In [ ]:
! pip install -r requirements.txt

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    pred_list = []
    target_list = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

            pred_list.extend(torch.flatten(pred).tolist())
            target_list.extend(target.tolist())

    test_loss /= len(test_loader.dataset)


    pred_list = pd.Series(pred_list, name='Actual')
    target_list = pd.Series(target_list, name='Predicted')
    cm = pd.crosstab(target_list, pred_list)

    print(cm)
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [ ]:
use_cuda = True
use_mps = False
test_batch_size = 1000

use_cuda = not use_cuda and torch.cuda.is_available()
use_mps = not use_mps and torch.backends.mps.is_available()

if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

test_kwargs = {"batch_size": test_batch_size}
if use_cuda:
    cuda_kwargs = {"num_workers": 1, "pin_memory": True, "shuffle": True}
    test_kwargs.update(cuda_kwargs)

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)
dataset2 = datasets.MNIST("../data", train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)



In [ ]:
model = Net().to(device)
model.load_state_dict(torch.utils.model_zoo.load_url("https://rsekube.nrp-nautilus.io/mnist_cnn.pt", model_dir=".", map_location=device))
test(model, device, test_loader)

In [ ]:
fig, ax = plt.subplots(1, 10, figsize=(20, 2))
for idx,(test_images, test_labels) in enumerate(test_loader):
    ax[idx].imshow(test_images[0][0], cmap='gray')
    ax[idx].set_title(f'{test_labels[0]}')
    ax[idx].axis('off')
    if idx == len(ax)-1:
        break